In [1]:
import pickle
import pandas as pd
import os

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm, trange
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch

from sklearn.preprocessing import MinMaxScaler
import math
from torch.profiler import profile, record_function, ProfilerActivity

from operator import itemgetter
import operator
from random import randint
from rnn_classes import Dog, DogInput, Race, Races, GRUNet, smallGRUNet, smalll_lin_GRUNet
from raceDB import build_dataset, build_pred_dataset
import importlib
import datetime
from training_testing import validate_model, train_regular, train_log, train_super_batch, train_super_batch_KL, train_super_batch_L1, train_regular_L1,train_regular_one_hot
from model_saver import model_saver, model_saver_wandb

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [73]:
all_price = pd.read_excel(r'C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA\model_all_price\prime-sweep-8 - all_price_df.xlsx')

In [22]:
all_price = pd.read_excel(r'C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA\model_all_price\forthright-quiver-153 - all_price_df.xlsx')

In [2]:
os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
all_data = pd.read_pickle('new_windows_gru_REAL.npy')

In [5]:
raceDB = Races(hidden_size=64,layers=2)

In [124]:


dog_stats_df = all_data
grouped = dog_stats_df.groupby(['track_name','dist'], as_index=False)

grouped_track_box = dog_stats_df.groupby(['track_name', 'box', 'dist'], as_index=False)
x =grouped_track_box.margin.sum()
track_margin_sum = grouped.margin.sum().reset_index()
x_r = x.reset_index().merge(track_margin_sum, how='left', on=['track_name','dist'])
x_r['adj'] = x_r['margin_x']/x_r['margin_y']
x_r_g = x_r.groupby(['track_name','dist'], as_index=False)
margin_weights = {}
for i,j in x_r_g:
    test = j
    #break
    track = i[0]
    dist = i[1]
    #margin_weights[track] = {}
    weights = j['adj'].tolist()
    if len(weights)!= 8:
        weights.append(0)
    margin_weights[i]  = torch.tensor(weights).to('cuda:0')# F.softmax()

In [125]:
margin_weights

{('Albion Park',
  331.0): tensor([0.1069, 0.1143, 0.1261, 0.1351, 0.1257, 0.1354, 0.1318, 0.1247],
        device='cuda:0'),
 ('Albion Park',
  395.0): tensor([0.1015, 0.1217, 0.1171, 0.1256, 0.1229, 0.1364, 0.1360, 0.1387],
        device='cuda:0'),
 ('Albion Park',
  520.0): tensor([0.1111, 0.1191, 0.1225, 0.1293, 0.1208, 0.1293, 0.1364, 0.1316],
        device='cuda:0'),
 ('Albion Park',
  600.0): tensor([0.1264, 0.1297, 0.1121, 0.1369, 0.1060, 0.1288, 0.1328, 0.1272],
        device='cuda:0'),
 ('Albion Park',
  710.0): tensor([0.1342, 0.1426, 0.0859, 0.1493, 0.0875, 0.1116, 0.1269, 0.1620],
        device='cuda:0'),
 ('Angle Park',
  342.0): tensor([0.1307, 0.1504, 0.0599, 0.1493, 0.1468, 0.0613, 0.1546, 0.1469],
        device='cuda:0'),
 ('Angle Park',
  388.0): tensor([0.1257, 0.1476, 0.0413, 0.1637, 0.1623, 0.0362, 0.1640, 0.1592],
        device='cuda:0'),
 ('Angle Park',
  515.0): tensor([0.1198, 0.1366, 0.0841, 0.1500, 0.1285, 0.0894, 0.1447, 0.1469],
        device='cuda:

In [109]:
x_r

,track_name,box,dist,margin_x,margin_y,adj
0,Albion Park,1.0,331.0,10025.20,344580.75,0.029094
1,Albion Park,1.0,395.0,10259.45,344580.75,0.029774
2,Albion Park,1.0,520.0,13946.50,344580.75,0.040474
3,Albion Park,1.0,600.0,2232.70,344580.75,0.006479
4,Albion Park,1.0,710.0,859.45,344580.75,0.002494
...,...,...,...,...,...,...
2012,Young,6.0,520.0,75.75,15203.80,0.004982
2013,Young,7.0,347.0,1859.10,15203.80,0.122279
2014,Young,7.0,520.0,106.25,15203.80,0.006988
2015,Young,8.0,347.0,1876.05,15203.80,0.123393


In [105]:
i

('Albion Park', 331.0)

In [85]:
all_data['box'] = all_data['stats'].apply(lambda x: x[0])
all_data

,dogid,dog_name,raceid,race_grade,date,trackOHE,track_name,state,track_code,dist,bfSP,runtime,prev_race,prev_race_date,place,margin,stats,box
0,314367490,SUPER BIG,501007024,Mixed 6/7,2019-12-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Healesville,VIC,306,350.0,5.481909,19.24,-1,-1,1.0,0.00,"[6.0, -1.0, 6.0, -1.0, -1.0, -1.0, -1.0, -1.0,...",6.0
1,141319750,SPRING SPINNER,500541116,Mixed 4/5,2019-12-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Mount Gambier,SA,504,600.0,18.096412,35.87,-1,-1,5.0,7.00,"[8.0, -1.0, 5.0, -1.0, -1.0, -1.0, -1.0, -1.0,...",8.0
2,240988036,MY GIRL COURT,500541116,Mixed 4/5,2019-12-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Mount Gambier,SA,504,600.0,110.000000,36.50,-1,-1,6.0,16.25,"[7.0, -1.0, 6.0, -1.0, -1.0, -1.0, -1.0, -1.0,...",7.0
3,312552897,BOTANY THOMMO,500543140,Class 1,2019-12-01,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Auckland,NZ,903,318.0,NaN,18.70,-1,-1,1.0,0.00,"[3.0, -1.0, 6.0, -1.0, -1.0, -1.0, -1.0, -1.0,...",3.0
4,295447218,FORD MAN,500543140,Class 1,2019-12-01,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Auckland,NZ,903,318.0,NaN,18.90,-1,-1,2.0,3.23,"[4.0, -1.0, 6.0, -1.0, -1.0, -1.0, -1.0, -1.0,...",4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1148894,485658949,KEEPING TUX,870434619,Grade 5 Heat,2023-02-14,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Mandurah,WA,601,405.0,NaN,23.77,864267587,2023-01-17 00:00:00,5.0,7.50,"[3.0, 31.3, 5.0, 0.05709754782254783, 0.058942...",3.0
1148895,679093555,WAFFLES MONELLI,870434619,Grade 5 Heat,2023-02-14,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Mandurah,WA,601,405.0,NaN,23.84,865622548,2023-01-27 00:00:00,6.0,8.25,"[8.0, 34.7, 5.0, 0.05685279854037841, 0.058641...",8.0
1148896,633121301,GRACIE LOU,870434619,Grade 5 Heat,2023-02-14,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Mandurah,WA,601,405.0,NaN,24.02,868957544,2023-02-04 00:00:00,7.0,11.00,"[7.0, 29.7, 5.0, 0.0579031994754217, 0.0592307...",7.0
1148897,654231358,MISS CONDOLEEZA,870434618,Grade 5 Heat,2023-02-14,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Mandurah,WA,601,405.0,NaN,23.09,870678530,2023-02-10 00:00:00,3.0,3.75,"[1.0, 28.2, 5.0, 0.0574363138278098, 0.0596578...",1.0


In [89]:
all_data_g = all_data.groupby(['track_name', 'dist', 'box'], as_index=False)

In [94]:
all_track_weights = all_data_g.margin.count()
all_track_weights

,track_name,dist,box,margin
0,Albion Park,331.0,1.0,1880
1,Albion Park,331.0,2.0,1857
2,Albion Park,331.0,3.0,1809
3,Albion Park,331.0,4.0,1843
4,Albion Park,331.0,5.0,1731
...,...,...,...,...
2012,Young,520.0,4.0,20
2013,Young,520.0,5.0,15
2014,Young,520.0,6.0,13
2015,Young,520.0,7.0,15


In [95]:
all_track_weights.to_csv('test weights 2.csv')

In [ ]:
track_dist = all_price.groupby(['flat_track'])

In [54]:
all_price = all_price[all_price['bet amount']!=0].copy()

In [51]:

profits = all_price.groupby(['flat_track', 'flat_date'], sort=False, as_index=False)[['profit < 30','outlay < 30']].sum()
profits

,flat_track,flat_date,profit < 30,outlay < 30
0,Bulli,2022-08-02,-212.440722,212.440722
1,Gosford,2022-08-02,12.083578,230.633561
2,Casino,2022-08-02,-189.571385,208.989630
3,Temora,2022-08-02,-80.645306,190.720290
4,Casino,2022-08-04,-51.365392,174.311254
...,...,...,...,...
595,Richmond,2023-02-08,265.876735,161.413862
596,The Gardens,2023-02-10,35.248913,193.478506
597,Richmond,2023-02-10,-9.436948,170.534209
598,Wagga Wagga,2023-02-10,251.332702,201.744455


In [ ]:
profits_roll = all_price.groupby(['flat_track', 'flat_date'], sort=False, as_index=False)[['profit < 30','outlay < 30']].rolling()

In [55]:
all_price['er'] = all_price.apply(lambda x: x['pred_prob']*(x['prices']-1), axis=1)
all_price['er']

0        3.666145
1        5.132390
2        1.224459
3        5.950561
5        7.491541
           ...   
46417    1.995933
46418    0.881184
46420    0.973830
46422    1.640015
46423    2.597663
Name: er, Length: 29283, dtype: float64

In [57]:
all_grouped = all_price.groupby(['flat_races'])

In [72]:
for i,j in all_grouped:
    j = j.sort_values(['er'], ascending=False)
    er_list = j['er']
    er_bet = [er_list[0]]

    R_s = 1
    bet_list = j.iloc[0:1]
    print(bet_list)
    for dog in j.iterrows():
        print(dog, "hey")
        if dog['er'] > R_s:
            bet_list.concat(dog)
        R_s = bet_list['pred_prob'].sum()/-(bet_list['prices']+1).sum()
        print(R_s)

        
    break

   Unnamed: 0  flat_dogs  flat_races     prices  pred_price  onehot_win  \
5           6  587949103   810746280  76.144858   10.030627           0   

   split_margin flat_track  flat_date  imp_prob  ...  frac kel  frac profit  \
5         10.75      Bulli 2022-08-02  0.013133  ...  8.787108    -8.787108   

    profit  flat_profit  colour  profit < 30 outlay < 30  fk profit < 30  \
5 -8.65618         -1.0    loss          0.0         0.0             0.0   

   fk outlay < 30        er  
5             0.0  7.491541  

[1 rows x 22 columns]
(5, Unnamed: 0                          6
flat_dogs                   587949103
flat_races                  810746280
prices                      76.144858
pred_price                  10.030627
onehot_win                          0
split_margin                    10.75
flat_track                      Bulli
flat_date         2022-08-02 00:00:00
imp_prob                     0.013133
pred_prob                    0.099695
bet amount                    8.

C:\Users\Nick\AppData\Local\Temp\ipykernel_22048\308645267.py:1: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for i,j in all_grouped:


TypeError: tuple indices must be integers or slices, not str